# LSTM Bot

## Project Overview

In this project, you will build a chatbot that can converse with you at the command line. The chatbot will use a Sequence to Sequence text generation architecture with an LSTM as it's memory unit. You will also learn to use pretrained word embeddings to improve the performance of the model. At the conclusion of the project, you will be able to show your chatbot to potential employers.

Additionally, you have the option to use pretrained word embeddings in your model. We have loaded Brown Embeddings from Gensim in the starter code below. You can compare the performance of your model with pre-trained embeddings against a model without the embeddings.



---



A sequence to sequence model (Seq2Seq) has two components:
- An Encoder consisting of an embedding layer and LSTM unit.
- A Decoder consisting of an embedding layer, LSTM unit, and linear output unit.

The Seq2Seq model works by accepting an input into the Encoder, passing the hidden state from the Encoder to the Decoder, which the Decoder uses to output a series of token predictions.

## Dependencies

- Pytorch
- Numpy
- Pandas
- NLTK
- Gzip
- Gensim


Please choose a dataset from the Torchtext website. We recommend looking at the Squad dataset first. Here is a link to the website where you can view your options:

- https://pytorch.org/text/stable/datasets.html





In [2]:
# Project Steps Overview and Estimated Duration
# Below you will find each of the components of the project, and estimated times to complete each portion. 
# These are estimates and not exact timings to help you expect the amount of time necessary to put aside to work on your project.

# Prepare data (~2 hours)
# Build your vocabulary from a corpus of language data. The Vocabulary object is described in Lesson Six: Seq2Seq.

# Build Model (~4 hours)
# Build your Encoder, Decoder, and larger Sequence to Sequence pattern in PyTorch. This pattern is described in Lesson Six: Seq2Seq.

# Train Model (~3 hours)
# Write your training procedure and divide your dataset into train/test/validation splits. Then, train your network and plot your evaluation metrics. Save your model after it reaches a satisfactory level of accuracy.

# Evaluate & Interact w/ Model (~1 hour)
# Write a script to interact with your network at the command line.

In [3]:
# Instructions Summary
# The LSTM Chatbot will help you show off your skills as a deep learning practitioner. You will develop the chatbot using a new architecture called a Seq2Seq. 
# Additionally, you can use pre-trained word embeddings to improve the performance of your model. Let's get started by following the steps below:

# Step 1: Build your Vocabulary & create the Word Embeddings
# The most important part of this step is to create your Vocabulary object using a corpus of data drawn from TorchText.

# (Extra Credit)
# Use Gensim to extract the word embeddings from one of its corpus'.
# Use NLTK and Gensim to create a function to clean your text and look up the index of a word's embeddings.

# Step 2: Create the Encoder
# A Seq2Seq architecture consists of an encoder and a decoder unit. You will use Pytorch to build a full Seq2Seq model.
# The first step of the architecture is to create an encoder with an LSTM unit.

# (Extra Credit)
# Load your pretrained embeddings into the LSTM unit.

# Step 3: Create the Decoder
# The second step of the architecture is to create a decoder using a second LSTM unit.

# Step 4: Combine them into a Seq2Seq Architecture
# To finalize your model, you will combine the encoder and decoder units into a working model.
# The Seq2Seq2 model must be able to instantiate the encoder and decoder. Then, it will accept the inputs for these units and manage their interaction to get an output using the forward pass function.

# Step 5: Train & evaluate your model
# Finally you will train and evaluate your model using a Pytorch training loop.

# Step 6: Interact with the Chatbot
# Demonstrate your chatbot by converting the outputs of the model to text and displaying it's responses at the command line.

In [1]:
# Pre-requisites: 
# - PyTorch 2.00 kernel
# - ml.g5.xlarge instance

# Install requirements
# !pip install gensim==4.3.1 nltk==3.8.1 torchtext torchdata portalocker | grep -v "already satisfied"

# !pip install gensim==4.3.1 nltk==3.8.1  torchtext==0.15.1 portalocker>=2.0.0 | grep -v "already satisfied"
!pip install gensim==4.3.1 nltk==3.8.1  torchtext==0.6.0   | grep -v "already satisfied"

#  torchtext==0.12.0 --> torch==1.11.0
#  torchtext==0.13.0 --> torch==1.12.0
#  torchtext==0.14.0 --> torch==1.12.0
# torchtext==0.15.1 --> torch==2.0.0
# torchtext==0.15.2 --> torch==2.0.1

# !pip install gensim==4.3.1 nltk==3.8.1  torchtext==0.10.0 | grep -v "already satisfied"

# !pip install gensim==4.2.0 nltk torchtext  | grep -v "already satisfied"


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


## Dataset load

In [2]:
import gensim
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
import torch.nn as nn
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
# import sklearn.model_selection 
from torchtext.utils import download_from_url
from torchtext.data import Field, BucketIterator, TabularDataset
import random
import json

nltk.download('brown')
nltk.download('punkt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Output, save, and load brown embeddings

# model = gensim.models.Word2Vec(brown.sents())
# model.save('brown.embedding')

# w2v = gensim.models.Word2Vec.load('brown.embedding')

question_context_field = Field(tokenize=word_tokenize, init_token='<sos>', eos_token='<eos>', lower=True)
answer_field = Field(tokenize=word_tokenize, init_token='<sos>', eos_token='<eos>', lower=True)

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
def prepare_text(sentence):
    return sentence

def loadDF(path):    
    data_file_path = download_from_url(path, root="data")
    
    with open(data_file_path, 'r') as f:
        squad_data = json.load(f)
        
    data = []
    examples = []
    for item in squad_data['data']:
        for paragraph in item['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']                
                # question_context = f"{question} {separator_token} {context}"
                question_context = f"{question} {context}"
                answer = qa['answers'][0]['text']
                
                data.append((question_context, answer))                                
                
    df = pd.DataFrame.from_records(data, columns=['question_context', 'answer'])        
    df['question_context'] = df['question_context'].apply(prepare_text)
    df['answer'] = df['answer'].apply(prepare_text)

    return df


def load_datasets(df, random_seed=42):
    df.to_csv("data/data.csv", index=False)
        
    fields = [('question_context', question_context_field), ('answer', answer_field)]
    dataset = TabularDataset("data/data.csv", format='csv', fields=fields)
    
    question_context_field.build_vocab(dataset, min_freq=1)
    answer_field.build_vocab(dataset, min_freq=1)

    train_data, valid_data = dataset.split(split_ratio=0.8, random_state=random.seed(random_seed))
    
    return train_data, valid_data


def decode_answer(tokens):
    return tokens_to_string(tokens, answer_field)

def encode_question(question_string):
    return string_to_tensor(question_string.lower(), question_context_field)
    
def tokens_to_string(tokens, field):
    eos_idx = field.vocab.stoi[field.eos_token]
    
    return " ".join([field.vocab.itos[token] for token in tokens if token != eos_idx])

def string_to_tensor(string, field):
    tokens = field.tokenize(string)

    tokens = ['<sos>'] + tokens + ['<eos>'] 

    token_ids = field.numericalize([tokens])
    
    return token_ids

    # tokens_to_string(token_ids.squeeze().tolist(), question_context_field)    
    
class PrintToBoth:
    def __init__(self, filename):
        self.file = open(filename, 'w')

    def print(self, s):
        print(s)
        print(s, file=self.file, flush=True)    

In [5]:
df = loadDF("https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json")

train_data, valid_data = load_datasets(df)

In [30]:
print(len(question_context_field.vocab))

print(len(answer_field.vocab))

102396
42794


In [31]:
df

,question_context,answer
0,To whom did the Virgin Mary allegedly appear i...,Saint Bernadette Soubirous
1,What is in front of the Notre Dame Main Buildi...,a copper statue of Christ
2,The Basilica of the Sacred heart at Notre Dame...,the Main Building
3,What is the Grotto at Notre Dame? Architectura...,a Marian place of prayer and reflection
4,What sits on top of the Main Building at Notre...,a golden statue of the Virgin Mary
...,...,...
87594,In what US state did Kathmandu first establish...,Oregon
87595,What was Yangon previously known as? Kathmandu...,Rangoon
87596,With what Belorussian city does Kathmandu have...,Minsk
87597,In what year did Kathmandu create its initial ...,1975


In [8]:
print(df.iloc[20]['question_context'])

print(df.iloc[20]['answer'])

What entity provides help with the management of time for new students at Notre Dame? All of Notre Dame's undergraduate students are a part of one of the five undergraduate colleges at the school or are in the First Year of Studies program. The First Year of Studies program was established in 1962 to guide incoming freshmen in their first year at the school before they have declared a major. Each student is given an academic advisor from the program who helps them to choose classes that give them exposure to any major in which they are interested. The program also includes a Learning Resource Center which provides time management, collaborative learning, and subject tutoring. This program has been recognized previously, by U.S. News & World Report, as outstanding.
Learning Resource Center


## Neural Network

In [9]:
import random

class Encoder(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, num_layers=1, dropout=0):
        
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.hidden = torch.zeros(1, 1, hidden_size)        # How to use it ????
        # self.embedding_dim = embedding_size
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)

        self.dropout = nn.Dropout(dropout)
    
    def forward(self, i):
        embedded = self.embedding(i)
        
        embedded = self.dropout(embedded)

        output, (hidden, cell) = self.lstm(embedded)
                
        return output, hidden, cell
    

class Decoder(nn.Module):
      
    def __init__(self, hidden_size, output_size, embedding_size, num_layers=1, dropout=0):
        
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding_size = embedding_size
        
        # self.embedding provides a vector representation of the target to our model
        # self.embedding = nn.Embedding(self.hidden_size, self.hidden_size)  # From lesson
        
        self.embedding = nn.Embedding(output_size, embedding_size)
        # self.embedding = nn.Embedding(hidden_size, embedding_size) # Why ?
        
        
        # self.lstm, accepts the embeddings and outputs a hidden state
        # self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, num_layers, dropout=(0 if num_layers == 1 else dropout))
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)

        # self.ouput, predicts on the hidden state via a linear output layer
        self.fc = nn.Linear(hidden_size, output_size)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        # print('[Decoder] input shape:', input.shape)
        
        input = input.unsqueeze(0)
        
        embedded = self.embedding(input)
        
        embedded = self.dropout(embedded)
            
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell)) 
        
        
        prediction = self.fc(output.squeeze(0))
        
        # print("prediction", prediction)
                
        return prediction, hidden, cell
        
        

class Seq2Seq(nn.Module):
        
    def __init__(self, encoder, decoder):
        
        super(Seq2Seq, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder        

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):              
        #src = [src len, batch size]
        #trg = [trg len, batch size]
                
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_size
                       
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(device)
        
        _, hidden, cell = self.encoder(src)
        
        # Start with <sos> tokens
        input = trg[0, :]
                
        for t in range(1, trg_len):

            output, hidden, cell = self.decoder(input, hidden, cell)

            outputs[t] = output
            
            # get highest predicted token
            top1 = output.argmax(1)
            
            # print("Most likely token: ", top1)
            
            use_teacher_forcing = random.random() < teacher_forcing_ratio

            input = trg[t] if use_teacher_forcing else top1
    
        return outputs


class Seq2SeqInference(nn.Module):
        
    def __init__(self, encoder, decoder, answer_field):
        
        super(Seq2SeqInference, self).__init__()
        
        self.encoder = encoder
        self.decoder = decoder        

    def forward(self, src, max_length=20):
        #src = [src len, 1]
        #trg = [trg len, 1]
        # expected output shape = [max_length, 1]
        
        if src.shape[1] != 1:
            raise ValueError(f"src.shape[1] != 1: {src.shape[1]}")
            
        batch_size = 1 # src.shape[1]

        trg_vocab_size = self.decoder.output_size
                                   
        logits = torch.zeros(max_length + 1, batch_size, trg_vocab_size).to(device)
        # outputs = torch.zeros(max_length + 1, batch_size).to(device)
        
        _, hidden, cell = self.encoder(src)
        
        # Start with <sos> tokens
        # TODO: create array of <sos> tokens      
        sos_idx = answer_field.vocab.stoi[answer_field.init_token] 
        
        # input = trg[0, :]
        
        # print("input shape", trg.shape)
        input = torch.tensor([sos_idx]).to(device)        
        # print("Start token: ", input)
        
        inferred_tokens = []
                
        # for t in range(1, trg_len):
        for t in range(1, max_length + 1):

            output, hidden, cell = self.decoder(input, hidden, cell)

            logits[t] = output
            
            # Get highest predicted token
            top1 = output.argmax(1)
            
            # print(f"Top1: {top1} {top1.item()}")
            # print(f"Most likely token: {top1.item()} ({answer_field.vocab.itos[top1.item()]})")
            inferred_tokens.append(top1)
                        
            input = top1                        
        
        inferred_tokens_tensor = torch.tensor(inferred_tokens).view(-1, 1)
        
        logits_dim = logits.shape[-1]
        logits = logits[1:].view(-1, logits_dim)
        
        return inferred_tokens_tensor, logits


## Model training

In [40]:
from torch.utils.data import DataLoader, TensorDataset

# Best hyperparameters: {'encoder_dropout': 0.6039111450304973, 'decoder_dropout': 0.33154046338693915, 'embedding_size': 468, 'hidden_size': 938, 'num_layers': 1, 'momentum': 0.5494925017261035, 'learning_rate': 0.04835390488901119, 'weight_decay': 0.00014738274625279338, 'batch_size': 132}
# Validation loss: 6.840339520820101

# Define the model and other parameters
encoder_input_size = len(question_context_field.vocab) 
encoder_embedding_size= 468 #256 
encoder_dropout = 0.6039111450304973 #0.1 

decoder_output_size = len(answer_field.vocab)
decoder_embedding_size = 256 #362 #300
decoder_dropout = 0.33154046338693915 #0.1 

hidden_size = 938 #256 #512 #1602
num_layers = 1
batch_size = 132 #64 #85 #128

learning_rate = 0.04835390488901119 #0.003 
weight_decay= 0.00014738274625279338 #1e-3
momentum= 0.5494925017261035 #0.5

num_epochs =  100

encoder = Encoder(encoder_input_size, hidden_size, encoder_embedding_size, num_layers, encoder_dropout)
decoder = Decoder(hidden_size, decoder_output_size, decoder_embedding_size, num_layers, decoder_dropout)
    
model = Seq2Seq(encoder, decoder).to(device)

train_iterator, valid_iterator = BucketIterator.splits(
    (train_data, valid_data),
   batch_size=batch_size,
   sort_within_batch=True,
    sort_key = lambda x: len(x.question_context),
    device=device)


# Init model weights
for name, param in model.named_parameters():
    nn.init.uniform_(param.data, -0.08, 0.08)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(102396, 468)
    (lstm): LSTM(468, 938, dropout=0.6039111450304973)
    (dropout): Dropout(p=0.6039111450304973, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(42794, 256)
    (lstm): LSTM(256, 938, dropout=0.33154046338693915)
    (fc): Linear(in_features=938, out_features=42794, bias=True)
    (dropout): Dropout(p=0.33154046338693915, inplace=False)
  )
)

In [41]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
        
now = datetime.datetime.now()
train_output_filename = f'train_output_{now.strftime("%Y%m%d_%H%M%S")}.txt'        
tee_print = PrintToBoth(train_output_filename)       

# learning_rate = 0.001
pad_idx = answer_field.vocab.stoi[answer_field.pad_token] 
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.01) # YOUR CODE HERE

# num_epochs =  20
clip = 5
valid_loss_min = None

for epoch in range(1, num_epochs + 1):
    start_time = time.time()
    model.train()
    train_loss = 0.0
    avg_train_loss = 0.0

    # Training loop
    for batch_idx, batch in enumerate(train_iterator):
        src = batch.question_context.to(device)
        trg = batch.answer.to(device)
        
#         print('src:', src.shape)
#         print('trg:', trg.shape)        
        
        optimizer.zero_grad()
        
        # print (src.shape)
        # print (trg.shape)
        
        # Pass the source sequences through the encoder
        output = model(src, trg)        #  [trg length, batch size, output dim]
        # print('output:', output.shape)
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
#         print('output 2:', output.shape)
#         print('trg 2:', trg.shape)
        
#         print('output content:', output)
#         print('trg content:', trg)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        train_loss += loss.item()
        
    average_train_loss = train_loss / len(train_iterator)
    
    end_time = time.time()    
    train_elapsed_time = end_time - start_time
    
    # Evaluation
    start_time = time.time()
    model.eval()
    with torch.no_grad():
        valid_loss = 0.0

        for batch_idx, batch in enumerate(valid_iterator):
            src = batch.question_context.to(device)
            trg = batch.answer.to(device)                    
            
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            valid_loss += loss.item()
            
        average_val_loss = valid_loss / len(valid_iterator)
                    
    end_time = time.time()    
    val_elapsed_time = end_time - start_time
    
    # print(f'Eval Time: {elapsed_time}')
    # print(f'Epoch: {epoch+1:02} | Eval Time: {val_elapsed_time}s')
    scheduler.step(average_val_loss)    
    
    torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, f'checkpoints/train-{epoch}.pt')
    
    if valid_loss_min is None or (
            (valid_loss_min - average_val_loss) / valid_loss_min > 0.01   # valid_loss_min - 0.01 * valid_loss_min    > average_val_loss
    ):
        # print(f"New minimum validation loss: {average_val_loss:.6f}. Saving model ...")
        tee_print.print(f"New minimum validation loss: {average_val_loss:.6f}. Saving model ...")
        
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': train_loss,
            }, 'checkpoints/best_val_loss.pt')

        valid_loss_min = average_val_loss
        
    tee_print.print(f"Epoch: {epoch:02}, Train Loss: {average_train_loss:.3f}, Val Loss: {average_val_loss:.3f} Train Time: {train_elapsed_time}s Eval Time: {val_elapsed_time}s")



New minimum validation loss: 7.164206. Saving model ...
Epoch: 01, Train Loss: 7.660, Val Loss: 7.164 Train Time: 188.57793474197388s Eval Time: 20.22123908996582s
New minimum validation loss: 6.980491. Saving model ...
Epoch: 02, Train Loss: 7.049, Val Loss: 6.980 Train Time: 187.96233129501343s Eval Time: 20.14483880996704s
New minimum validation loss: 6.890448. Saving model ...
Epoch: 03, Train Loss: 6.922, Val Loss: 6.890 Train Time: 189.05081629753113s Eval Time: 20.188480854034424s
Epoch: 04, Train Loss: 6.864, Val Loss: 6.863 Train Time: 189.4222068786621s Eval Time: 20.15251111984253s
Epoch: 05, Train Loss: 6.825, Val Loss: 6.844 Train Time: 189.15589547157288s Eval Time: 20.124694347381592s
Epoch: 06, Train Loss: 6.800, Val Loss: 6.846 Train Time: 189.11357736587524s Eval Time: 20.227267742156982s
New minimum validation loss: 6.820378. Saving model ...
Epoch: 07, Train Loss: 6.783, Val Loss: 6.820 Train Time: 188.4856460094452s Eval Time: 20.12617588043213s
Epoch: 08, Train Lo

KeyboardInterrupt: 

In [ ]:
# New minimum validation loss: 9.777593. Saving model ...
# Epoch: 01, Train Loss: 10.325, Val Loss: 9.778 Train Time: 138.35026669502258s Eval Time: 15.636022806167603s
# New minimum validation loss: 8.429811. Saving model ...
# Epoch: 02, Train Loss: 8.829, Val Loss: 8.430 Train Time: 137.56066799163818s Eval Time: 15.643161058425903s
# New minimum validation loss: 8.051929. Saving model ...
# Epoch: 03, Train Loss: 8.262, Val Loss: 8.052 Train Time: 137.84890174865723s Eval Time: 15.51107907295227s
# New minimum validation loss: 7.829260. Saving model ...
# Epoch: 04, Train Loss: 7.918, Val Loss: 7.829 Train Time: 138.03220987319946s Eval Time: 15.569308996200562s
# New minimum validation loss: 7.725741. Saving model ...
# Epoch: 05, Train Loss: 7.784, Val Loss: 7.726 Train Time: 138.2921051979065s Eval Time: 15.532844543457031s
# New minimum validation loss: 7.636809. Saving model ...
# Epoch: 06, Train Loss: 7.686, Val Loss: 7.637 Train Time: 137.98424077033997s Eval Time: 15.647210597991943s
# Epoch: 07, Train Loss: 7.605, Val Loss: 7.561 Train Time: 138.06058549880981s Eval Time: 15.66235065460205s
# New minimum validation loss: 7.492020. Saving model ...
# Epoch: 08, Train Loss: 7.532, Val Loss: 7.492 Train Time: 137.3553535938263s Eval Time: 15.439005374908447s
# Epoch: 09, Train Loss: 7.467, Val Loss: 7.433 Train Time: 139.83897972106934s Eval Time: 15.809000015258789s
# New minimum validation loss: 7.379917. Saving model ...
# Epoch: 10, Train Loss: 7.412, Val Loss: 7.380 Train Time: 137.8189413547516s Eval Time: 15.560516834259033s
# Epoch: 11, Train Loss: 7.360, Val Loss: 7.332 Train Time: 136.65642929077148s Eval Time: 15.613484621047974s
# New minimum validation loss: 7.288950. Saving model ...
# Epoch: 12, Train Loss: 7.313, Val Loss: 7.289 Train Time: 137.94713592529297s Eval Time: 15.612228393554688s


## HyperParameter Optimization

In [12]:
!pip install  sqlalchemy==1.4.8  optuna | grep -v "already satisfied"


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from sqlalchemy import orm
import optuna
import time

encoder_input_size = len(question_context_field.vocab) 
decoder_output_size = len(answer_field.vocab)
pad_idx = answer_field.vocab.stoi[answer_field.pad_token] 

def objective(trial):
    torch.cuda.empty_cache()
    
    encoder_dropout = trial.suggest_float('encoder_dropout', 0.1, 0.7)
    decoder_dropout = trial.suggest_float('decoder_dropout', 0.1, 0.7)
    embedding_size = trial.suggest_int('embedding_size', 200, 512)
    hidden_size = trial.suggest_int('hidden_size', 64, 1024)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    momentum = trial.suggest_uniform('momentum', 0.1, 0.9)
        
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    # opt = trial.suggest_categorical('opt', ['sgd', 'adam'])
    # momentum = trial.suggest_uniform('momentum', 0.1, 0.9)
    
    batch_size = trial.suggest_int('batch_size', 32, 1024)
    
    print(f'Starting trial {trial.number}:\tHyperparameters={trial.params}') 
    
    train_iterator, valid_iterator = BucketIterator.splits(
        (train_data, valid_data),
       batch_size=batch_size,
       sort_within_batch=True,
        sort_key = lambda x: len(x.question_context),
        device=device)
    
    encoder = Encoder(encoder_input_size, hidden_size, embedding_size, num_layers, encoder_dropout)
    decoder = Decoder(hidden_size, decoder_output_size, embedding_size, num_layers, decoder_dropout)
    model = Seq2Seq(encoder, decoder).to(device)    
    
    # Init model weights
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
    
    criterion = nn.CrossEntropyLoss(ignore_index = pad_idx) # https://pytorch.org/docs/2.0/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss
    # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True, threshold=0.01) # YOUR CODE HERE
    
    num_epochs = 5
    clip = 5
    valid_loss_min = None

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        train_loss = 0.0
        avg_train_loss = 0.0

        # Training loop
        for batch_idx, batch in enumerate(train_iterator):
            src = batch.question_context.to(device)
            trg = batch.answer.to(device)

    #         print('src:', src.shape)
    #         print('trg:', trg.shape)        

            optimizer.zero_grad()


            # print (src.shape)
            # print (trg.shape)

            # Pass the source sequences through the encoder
            output = model(src, trg)        #  [trg length, batch size, output dim]
            # print('output:', output.shape)

            output_dim = output.shape[-1]

            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

    #         print('output 2:', output.shape)
    #         print('trg 2:', trg.shape)

    #         print('output content:', output)
    #         print('trg content:', trg)

            loss = criterion(output, trg)

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

            optimizer.step()

            # print(train_loss)

            train_loss += loss.item()


        average_train_loss = train_loss / len(train_iterator)

        end_time = time.time()    
        train_elapsed_time = end_time - start_time

        # print(f'Epoch: {epoch+1:02} | Train Time: {train_elapsed_time}s')

        # Evaluation on the test dataset
        start_time = time.time()
        model.eval()
        with torch.no_grad():
            valid_loss = 0.0

            for batch_idx, batch in enumerate(valid_iterator):
                src = batch.question_context.to(device)
                trg = batch.answer.to(device)                    

                output = model(src, trg, 0) #turn off teacher forcing

                #trg = [trg len, batch size]
                #output = [trg len, batch size, output dim]

                output_dim = output.shape[-1]

                output = output[1:].view(-1, output_dim)
                trg = trg[1:].view(-1)

                #trg = [(trg len - 1) * batch size]
                #output = [(trg len - 1) * batch size, output dim]

                loss = criterion(output, trg)

                valid_loss += loss.item()

            average_val_loss = valid_loss / len(valid_iterator)

        end_time = time.time()    
        val_elapsed_time = end_time - start_time

        # print(f'Eval Time: {elapsed_time}')
        # print(f'Epoch: {epoch+1:02} | Eval Time: {val_elapsed_time}s')

        scheduler.step(average_val_loss)
        
        if valid_loss_min is None or (
                (valid_loss_min - average_val_loss) / valid_loss_min > 0.01
        ):
            print(f"New minimum validation loss: {average_val_loss:.6f}. Saving model ...")

            torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': train_loss,
                }, f'checkpoints/hpo_{trial.number}_model.pt",')

            valid_loss_min = average_val_loss

        print(f"Epoch: {epoch+1:02}, Train Loss: {average_train_loss:.3f}, Val Loss: {average_val_loss:.3f} Train Time: {train_elapsed_time}s Eval Time: {val_elapsed_time}s")    
        
    return valid_loss_min
    
# Define study and optimize hyperparameters
study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=100, catch=(torch.cuda.OutOfMemoryError,))

[I 2023-11-18 23:08:48,921] A new study created in memory with name: no-name-1f73075e-d6d6-42e0-bb28-89e1c8bbfc17
/tmp/ipykernel_537/4151047639.py:17: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  momentum = trial.suggest_uniform('momentum', 0.1, 0.9)
/tmp/ipykernel_537/4151047639.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)


Starting trial 0:	Hyperparameters={'encoder_dropout': 0.10507486873471816, 'decoder_dropout': 0.40833098697311565, 'embedding_size': 240, 'hidden_size': 932, 'num_layers': 2, 'momentum': 0.41814445054158134, 'learning_rate': 0.00011683107152760196, 'weight_decay': 0.0037540186215223427, 'batch_size': 71}
New minimum validation loss: 10.646195. Saving model ...
Epoch: 01, Train Loss: 10.651, Val Loss: 10.646 Train Time: 237.41277265548706s Eval Time: 22.522602796554565s
Epoch: 02, Train Loss: 10.635, Val Loss: 10.624 Train Time: 237.06071305274963s Eval Time: 22.41866135597229s
Epoch: 03, Train Loss: 10.615, Val Loss: 10.600 Train Time: 235.52900218963623s Eval Time: 22.411000728607178s
Epoch: 04, Train Loss: 10.595, Val Loss: 10.575 Train Time: 235.67363333702087s Eval Time: 22.463497400283813s


[I 2023-11-18 23:30:33,565] Trial 0 finished with value: 10.646194859554893 and parameters: {'encoder_dropout': 0.10507486873471816, 'decoder_dropout': 0.40833098697311565, 'embedding_size': 240, 'hidden_size': 932, 'num_layers': 2, 'momentum': 0.41814445054158134, 'learning_rate': 0.00011683107152760196, 'weight_decay': 0.0037540186215223427, 'batch_size': 71}. Best is trial 0 with value: 10.646194859554893.


Epoch: 05, Train Loss: 10.574, Val Loss: 10.549 Train Time: 237.03408646583557s Eval Time: 22.42497444152832s
Starting trial 1:	Hyperparameters={'encoder_dropout': 0.665348632640786, 'decoder_dropout': 0.687053155548927, 'embedding_size': 444, 'hidden_size': 782, 'num_layers': 2, 'momentum': 0.6949344175628585, 'learning_rate': 0.00899846356647276, 'weight_decay': 0.006562449189699982, 'batch_size': 790}


[W 2023-11-18 23:30:35,325] Trial 1 failed with parameters: {'encoder_dropout': 0.665348632640786, 'decoder_dropout': 0.687053155548927, 'embedding_size': 444, 'hidden_size': 782, 'num_layers': 2, 'momentum': 0.6949344175628585, 'learning_rate': 0.00899846356647276, 'weight_decay': 0.006562449189699982, 'batch_size': 790} because of the following error: OutOfMemoryError('CUDA out of memory. Tried to allocate 3.90 GiB (GPU 0; 22.20 GiB total capacity; 18.99 GiB already allocated; 141.12 MiB free; 20.83 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_537/4151047639.py", line 83, in objective
    loss = criterion(output, trg)
  File "/opt/conda/lib/

...

New minimum validation loss: 7.281041. Saving model ...
Epoch: 01, Train Loss: 7.809, Val Loss: 7.281 Train Time: 190.14013409614563s Eval Time: 19.358969926834106s
New minimum validation loss: 7.045174. Saving model ...
Epoch: 02, Train Loss: 7.126, Val Loss: 7.045 Train Time: 188.74524021148682s Eval Time: 19.301857709884644s
Epoch: 03, Train Loss: 6.984, Val Loss: 6.979 Train Time: 189.23022866249084s Eval Time: 19.335997819900513s
New minimum validation loss: 6.916367. Saving model ...
Epoch: 04, Train Loss: 6.911, Val Loss: 6.916 Train Time: 190.07164096832275s Eval Time: 19.33453106880188s


[I 2023-11-19 00:42:04,122] Trial 26 finished with value: 6.916367074204839 and parameters: {'encoder_dropout': 0.666860866556084, 'decoder_dropout': 0.14310546756612483, 'embedding_size': 403, 'hidden_size': 686, 'num_layers': 1, 'momentum': 0.5304037074535177, 'learning_rate': 0.016803731754976535, 'weight_decay': 0.007561913049800378, 'batch_size': 62}. Best is trial 26 with value: 6.916367074204839.


Epoch: 05, Train Loss: 6.869, Val Loss: 6.885 Train Time: 188.61841201782227s Eval Time: 19.39446473121643s
Starting trial 27:	Hyperparameters={'encoder_dropout': 0.21880156466541706, 'decoder_dropout': 0.36376582554937464, 'embedding_size': 280, 'hidden_size': 960, 'num_layers': 1, 'momentum': 0.36099383777669003, 'learning_rate': 0.00032189008410052556, 'weight_decay': 4.067770746713994e-06, 'batch_size': 242}


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.21880156466541706 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.36376582554937464 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


New minimum validation loss: 10.654181. Saving model ...
Epoch: 01, Train Loss: 10.660, Val Loss: 10.654 Train Time: 171.24145770072937s Eval Time: 19.288434982299805s


In [19]:
print('Best trial:', study.best_trial)
print('Best hyperparameters:', study.best_params)
print('Validation loss:', study.best_value)

Best trial: FrozenTrial(number=31, state=TrialState.COMPLETE, values=[6.840339520820101], datetime_start=datetime.datetime(2023, 11, 19, 1, 42, 47, 67924), datetime_complete=datetime.datetime(2023, 11, 19, 2, 0, 58, 397423), params={'encoder_dropout': 0.6039111450304973, 'decoder_dropout': 0.33154046338693915, 'embedding_size': 468, 'hidden_size': 938, 'num_layers': 1, 'momentum': 0.5494925017261035, 'learning_rate': 0.04835390488901119, 'weight_decay': 0.00014738274625279338, 'batch_size': 132}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'encoder_dropout': FloatDistribution(high=0.7, log=False, low=0.1, step=None), 'decoder_dropout': FloatDistribution(high=0.7, log=False, low=0.1, step=None), 'embedding_size': IntDistribution(high=512, log=False, low=200, step=1), 'hidden_size': IntDistribution(high=1024, log=False, low=64, step=1), 'num_layers': IntDistribution(high=3, log=False, low=1, step=1), 'momentum': FloatDistribution(high=0.9, log=False, low=0.1, s

In [ ]:
# [I 2023-11-19 00:01:53,886] Trial 13 finished with value: 8.002150303880933 and parameters: {'encoder_dropout': 0.22955902660914207, 'decoder_dropout': 0.4911158328281042, 'embedding_size': 491, 'hidden_size': 975, 'num_layers': 1, 'momentum': 0.6972683887927308, 'learning_rate': 0.0006676334684955292, 'weight_decay': 0.0033224254292016676, 'batch_size': 124}. Best is trial 13 with value: 8.002150303880933.


# [I 2023-11-19 00:22:13,189] Trial 19 finished with value: 7.340739211429556 and parameters: {'encoder_dropout': 0.18872017707563837, 'decoder_dropout': 0.26471499196465104, 'embedding_size': 345, 'hidden_size': 809, 'num_layers': 1, 'momentum': 0.8770181216883318, 'learning_rate': 0.0006037527750744888, 'weight_decay': 1.0067442886238327e-06, 'batch_size': 62}. Best is trial 19 with value: 7.340739211429556.

# [I 2023-11-19 00:42:04,122] Trial 26 finished with value: 6.916367074204839 and parameters: {'encoder_dropout': 0.666860866556084, 'decoder_dropout': 0.14310546756612483, 'embedding_size': 403, 'hidden_size': 686, 'num_layers': 1, 'momentum': 0.5304037074535177, 'learning_rate': 0.016803731754976535, 'weight_decay': 0.007561913049800378, 'batch_size': 62}. Best is trial 26 with value: 6.916367074204839.


# Best hyperparameters: {'encoder_dropout': 0.6039111450304973, 'decoder_dropout': 0.33154046338693915, 'embedding_size': 468, 'hidden_size': 938, 'num_layers': 1, 'momentum': 0.5494925017261035, 'learning_rate': 0.04835390488901119, 'weight_decay': 0.00014738274625279338, 'batch_size': 132}
# Validation loss: 6.840339520820101



## Running Inference

In [42]:
test_encoder = Encoder(encoder_input_size, hidden_size, encoder_embedding_size, num_layers, encoder_dropout)
test_decoder = Decoder(hidden_size, decoder_output_size, decoder_embedding_size, num_layers, decoder_dropout)
    
inference_model = Seq2SeqInference(test_encoder, test_decoder, answer_field)
inference_model.to(device)

pad_idx = answer_field.vocab.stoi[answer_field.pad_token] 
test_criterion = nn.CrossEntropyLoss(ignore_index = pad_idx) # https://pytorch.org/docs/2.0/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss
test_criterion.to(device)

# checkpoint = torch.load('checkpoints/best_val_loss.pt')
checkpoint = torch.load('checkpoints/train-25.pt')
inference_model.load_state_dict(checkpoint['model_state_dict'])


test_iterator = BucketIterator(
        # dataset=valid_data,
        dataset=train_data,
       batch_size=1,
       sort_within_batch=True,
        sort_key = lambda x: len(x.question_context),
        device=device)


In [46]:
# Smoke text

with torch.no_grad():
    valid_loss = 0.0

    for batch_idx, batch in enumerate(test_iterator):
        src = batch.question_context.to(device)
        trg = batch.answer.to(device)
        
        # print("src shape", src.shape)
        # print("src", src)
        # print("trg", trg)
        
        tokens, logits = inference_model(src, 5)

        #trg = [trg len, batch size]
        #logits = [max length, batch, output dim]

#         print("logits shape", logits.shape)
#         print("seq2seq-logits-raw", logits)
        
#         logits_dim = logits.shape[-1]
#         logits = logits[1:].view(-1, logits_dim)
        
        # trg = trg[1:].view(-1)

        # print("seq2seq-logits", logits)
        # print("seq2seq-trg", trg)
        # print("tokens shape", tokens.shape)
        # print("tokens", tokens)
                
        # response_text = tokens_to_string(tokens.squeeze().tolist())
        response_text = decode_answer(tokens.squeeze().tolist())
        
        print(response_text)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]

        # TODO: question: should 'output' really be the MLP result tensor ?
#         loss = test_criterion(output, trg)

#         valid_loss += loss.item()
        
        break # TODO: remove line





the


## Chatbot experience

In [47]:
def respond_question(question):
    with torch.no_grad():
        question_tensor = encode_question(question)

        tokens, logits = inference_model(src, 30)

        response_text = decode_answer(tokens.squeeze().tolist())

        return response_text
    

In [48]:
respond_question("A week comprise seven days. What day do you prefer ?")


'the'

In [49]:
respond_question("What would change the rotational inertia of a body under Newton's First Law of Motion? Torque is the rotation equivalent of force in the same way that angle is the rotational equivalent for position, angular velocity for velocity, and angular momentum for momentum. As a consequence of Newton's First Law of Motion, there exists rotational inertia that ensures that all bodies maintain their angular momentum unless acted upon by an unbalanced torque. Likewise, Newton's Second Law of Motion can be used to derive an analogous equation for the instantaneous angular acceleration of the rigid body")

'the'

In [ ]:
respond_question("What would change the rotational inertia of a body under Newton's First Law of Motion? Torque is the rotation equivalent of force in the same way that angle is the rotational equivalent for position, angular velocity for velocity, and angular momentum for momentum. As a consequence of Newton's First Law of Motion, there exists rotational inertia that ensures that all bodies maintain their angular momentum unless acted upon by an unbalanced torque. Likewise, Newton's Second Law of Motion can be used to derive an analogous equation for the instantaneous angular acceleration of the rigid body")

In [ ]:
respond_question("What changes macroscopic closed system energies? The connection between macroscopic nonconservative forces and microscopic conservative forces is described by detailed treatment with statistical mechanics. In macroscopic closed systems, nonconservative forces act to change the internal energies of the system, and are often associated with the transfer of heat. According to the Second law of thermodynamics, nonconservative forces necessarily result in energy transformations within closed systems from ordered to more random conditions as entropy increases.")